In [1]:
import openpyxl
import pandas as pd

#With openpyxl, we start an empty workbook first
wb = openpyxl.Workbook()

# grab the active worksheet
ws = wb.active

# Write data to cell by the letter number combination
ws['A1'] = 42

#Write data to cell by the row/column combination
ws.cell(row=1,column=5).value = 5
ws.cell(row=5,column=1).value = 5

# Save the file
import os
if not os.path.exists("Workbooks"):
    os.mkdir("Workbooks")
wb.save("Workbooks/Example.xlsx")

In [2]:
#Read in table
yield_curve_table = pd.read_csv("Tables/Treasury Yield Table.csv",index_col=0)
yield_curve_table

,Current,1 Year Ago Curve,1 Year Change,5 Years Ago Curve,5 Year Change
1 Mo,1.48,2.44,-0.96,0.03,1.45
3 Mo,1.55,2.45,-0.90,0.04,1.51
6 Mo,1.60,2.56,-0.96,0.12,1.48
1 Yr,1.59,2.63,-1.04,0.25,1.34
2 Yr,1.58,2.48,-0.90,0.67,0.91
3 Yr,1.62,2.46,-0.84,1.10,0.52
5 Yr,1.69,2.51,-0.82,1.65,0.04
7 Yr,1.83,2.59,-0.76,1.97,-0.14
10 Yr,1.92,2.69,-0.77,2.17,-0.25
20 Yr,2.25,2.87,-0.62,2.47,-0.22


In [3]:
#Doing values.tolist() on a dataframe returns the following
print(yield_curve_table.reset_index().values.tolist())

[['1 Mo', 1.48, 2.44, -0.96, 0.03, 1.45], ['3 Mo', 1.55, 2.45, -0.9000000000000001, 0.04, 1.51], ['6 Mo', 1.6, 2.56, -0.96, 0.12, 1.48], ['1 Yr', 1.59, 2.63, -1.0399999999999998, 0.25, 1.34], ['2 Yr', 1.58, 2.48, -0.8999999999999999, 0.67, 0.91], ['3 Yr', 1.62, 2.46, -0.8399999999999999, 1.1, 0.52], ['5 Yr', 1.69, 2.51, -0.8199999999999998, 1.65, 0.040000000000000036], ['7 Yr', 1.83, 2.59, -0.7599999999999998, 1.97, -0.1399999999999999], ['10 Yr', 1.92, 2.69, -0.77, 2.17, -0.25], ['20 Yr', 2.25, 2.87, -0.6200000000000001, 2.47, -0.2200000000000002], ['30 Yr', 2.39, 3.02, -0.6299999999999999, 2.75, -0.3599999999999999]]


In [4]:
#Define a function to write a pandas dataframe to excel
def write_to_excel(ws, start_row, start_column, df, index=True, columns = True):
    #If we want to include the index, reset the index on the dataframe first
    if index:
        write_data = df.reset_index().values.tolist()
    else:
        write_data = df.values.tolist()
    
    #If we want to include columns we need to add the columns as the first in the nested lists
    if columns:
        #If we already added an index, then we are going to want also include a blank space since
        #we don't want a label above the index
        if index:
            write_data = [[""]+list(df.columns)]+write_data
        else:
            write_data = [list(df.columns)]+write_data
    
    #For every value iterate through finding the location to place it in
    for row in range(len(write_data)):
        for column in range(len(write_data[0])):
            ws.cell(row=start_row+row,column=start_column+column).value = write_data[row][column]

wb = openpyxl.Workbook()
ws = wb.active

#Examples of writing to excel
write_to_excel(ws, 1, 1, yield_curve_table)
write_to_excel(ws, 16, 1, yield_curve_table,index=False)
write_to_excel(ws, 31, 1, yield_curve_table,columns=False)
write_to_excel(ws, 46, 1, yield_curve_table,index=False,columns=False)

wb.save("Workbooks/Example 2.xlsx")

In [5]:
from openpyxl.styles import PatternFill

wb = openpyxl.Workbook()
ws = wb.active

write_to_excel(ws, 3, 1, yield_curve_table)
#PatternFill is a way to get colors, here we pass the start and end color as the same and use a fill type solid
my_fill_color = PatternFill(start_color='FF0000',
                   end_color='FF0000',
                   fill_type='solid')

#The fill attribute of a cell is the color to use
ws.cell(row=1,column=1).fill = my_fill_color
wb.save("Workbooks/Example 3.xlsx")

In [6]:
#Create a function to map red and green colors based on whether a field is positive or negative
def red_green_map(df, start_row, start_column):
    write_data = df.values.tolist()
    for row in range(len(write_data)):
        for column in range(len(write_data[0])):
            if write_data[row][column] > 0:
                fill = fill = PatternFill(start_color='008000',
                   end_color='008000',
                   fill_type='solid')
            elif write_data[row][column] < 0:
                fill = PatternFill(start_color='FF0000',
                   end_color='FF0000',
                   fill_type='solid')
            ws.cell(row=start_row+row,column=start_column+column).fill = fill


wb = openpyxl.Workbook()
ws = wb.active

#Write the data to excel
write_to_excel(ws, 3, 1, yield_curve_table)
#We'll deal with only the two change columns
#So pass the data for only those and also pass their starting columns/rows
red_green_map(yield_curve_table[["1 Year Change"]], 4, 4)
red_green_map(yield_curve_table[["5 Year Change"]], 4, 6)

wb.save("Workbooks/Example 4.xlsx")

In [7]:
#Put it all together for a function that takes a worksheet and yield curve data then writes the sheet
def write_treasury_sheet(ws, yield_curve_table):
    #Write the yield curve table
    write_to_excel(ws, 3, 1, yield_curve_table)
    #Color the yield curve changes red/green
    red_green_map(yield_curve_table[["1 Year Change"]], 4, 4)
    red_green_map(yield_curve_table[["5 Year Change"]], 4, 6)
    
wb = openpyxl.Workbook()
ws = wb.active
write_treasury_sheet(ws, yield_curve_table)
wb.save("Workbooks/Economic Report.xlsx")

In [8]:
from openpyxl.utils.cell import get_column_letter
#The get_column_letter function will convert a number to a letter for excel
print(get_column_letter(1))

A


In [9]:
#If we choose a specific column through ws.column_dimensions, we can set the width
def adjust_columns(ws,start,values):
    for i, width in enumerate(values):
        ws.column_dimensions[get_column_letter(start+i)].width = width

wb = openpyxl.Workbook()
ws = wb.active

write_to_excel(ws, 3, 1, yield_curve_table)

adjust_columns(ws,1,[2,10,20,5])

wb.save("Workbooks/Example 5.xlsx")

In [10]:
#Update the function
def write_treasury_sheet(ws, yield_curve_table):
    #Write the yield curve table
    write_to_excel(ws, 3, 1, yield_curve_table)
    
    #Color the yield curve changes red/green
    red_green_map(yield_curve_table[["1 Year Change"]], 4, 4)
    red_green_map(yield_curve_table[["5 Year Change"]], 4, 6)
    
    #Adjust the column widths for the table
    adjust_columns(ws,2,[15]*len(yield_curve_table.columns))
    
wb = openpyxl.Workbook()
ws = wb.active
write_treasury_sheet(ws, yield_curve_table)
wb.save("Workbooks/Economic Report.xlsx")

In [11]:
from openpyxl.styles import Font


wb = openpyxl.Workbook()
ws = wb.active



#We can import Font and set one like so
write_to_excel(ws, 3, 1, yield_curve_table)
font = Font(bold=True)
ws.cell(row=4,column=1).font = font

#Or we can take the current fond and modify it like so to make it bold
font = ws.cell(row=3,column=2).font.copy()
print(font)
font.bold = True
ws.cell(row=3,column=2).font = font

wb.save("Workbooks/Example 6.xlsx")

<openpyxl.styles.fonts.Font object>
Parameters:
name='Calibri', charset=None, family=2.0, b=False, i=False, strike=None, outline=None, shadow=None, condense=None, color=<openpyxl.styles.colors.Color object>
Parameters:
rgb=None, indexed=None, auto=None, theme=1, tint=0.0, type='theme', extend=None, sz=11.0, u=None, vertAlign=None, scheme='minor'


/Users/seanmcowen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
  from ipykernel import kernelapp as app


In [12]:
#A function to bold everything in either a vertical or horizontal manner given a length of cells to bold
#and starting position
def bold_cells_line(ws, start_row, start_column, length, vertical=True):
    for i in range(length):
        if vertical:
            cell = ws.cell(row=start_row+i,column=start_column)
        else:
            cell = ws.cell(row=start_row,column=start_column+i)
        font = cell.font.copy()
        font.bold = True
        cell.font = font

def write_treasury_sheet(ws, yield_curve_table):
    #Write the yield curve table
    write_to_excel(ws, 3, 1, yield_curve_table)
    
    #Color the yield curve changes red/green
    red_green_map(yield_curve_table[["1 Year Change"]], 4, 4)
    red_green_map(yield_curve_table[["5 Year Change"]], 4, 6)
    
    #Adjust the column widths for the table
    adjust_columns(ws,2,[15]*len(yield_curve_table.columns))
    
    #Bold the index and columns
    bold_cells_line(ws, 4, 1, len(yield_curve_table), vertical=True)
    bold_cells_line(ws, 3, 2, len(yield_curve_table.columns), vertical=False)
    
wb = openpyxl.Workbook()
ws = wb.active
write_treasury_sheet(ws, yield_curve_table)
wb.save("Workbooks/Economic Report.xlsx")

/Users/seanmcowen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
  if __name__ == '__main__':


In [13]:
wb = openpyxl.Workbook()
ws = wb.active

write_to_excel(ws, 3, 1, yield_curve_table)

#Merging cells can be done with ws.merge_cells
ws.merge_cells('A1:F1')

wb.save("Workbooks/Example 7.xlsx")

In [14]:
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string

#coordinate_from_string returns the coordinates for the string representation of a cell  
print(coordinate_from_string("B6"))

#column_index_from_string converts a letter to the actual number in the excel sheet for a coordinate
print(column_index_from_string("B"))

('B', 6)
2


In [15]:
from openpyxl.styles.alignment import Alignment

#A title function for the worksheet
def create_title(ws, area, text):
    ws.merge_cells(area)
    #Given an area such as A1:A5, we want to first isolate the first cell
    first_cell = area.split(":")[0]
    #And then find the numerical row and column associated
    first_cell = coordinate_from_string(first_cell)
    col = column_index_from_string(first_cell[0])
    row = first_cell[1]
    
    #Change the text of the merged cell to whatever was passed as input
    ws.cell(row=row, column=col).value = text
    #Change the font of the title
    font = font = Font(size=16,
                       bold=True,
                      color="283747")
    ws.cell(row=row, column=col).font = font
    #Change alignment to be centered
    ws.cell(row=row, column=col).alignment = Alignment(horizontal="center")

#Update the function
def write_treasury_sheet(ws, yield_curve_table):
    #Write the yield curve table
    write_to_excel(ws, 3, 1, yield_curve_table)
    
    #Color the yield curve changes red/green
    red_green_map(yield_curve_table[["1 Year Change"]], 4, 4)
    red_green_map(yield_curve_table[["5 Year Change"]], 4, 6)
    
    #Adjust the column widths for the table
    adjust_columns(ws,2,[15]*len(yield_curve_table.columns))
    
    #Bold the index and columns
    bold_cells_line(ws, 4, 1, len(yield_curve_table), vertical=True)
    bold_cells_line(ws, 3, 2, len(yield_curve_table.columns), vertical=False)
    
    #Create title
    create_title(ws, 'A1:F1', "Treasury Yield Curve")
    
wb = openpyxl.Workbook()
ws = wb.active
write_treasury_sheet(ws, yield_curve_table)
wb.save("Workbooks/Economic Report.xlsx")

/Users/seanmcowen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
  if __name__ == '__main__':


In [16]:
from openpyxl.styles import Border, Side

wb = openpyxl.Workbook()
ws = wb.active


write_to_excel(ws, 3, 1, yield_curve_table)

#With borders we need to speicify the styles we want to use for each border
border = Border(left=Side(border_style='thin'),
         right=Side(border_style='thin'),
         top=Side(border_style='thin'),
         bottom=Side(border_style='thin'))
temp = ws.cell(row=5,column=5)
temp.border = border
wb.save("Workbooks/Example 8.xlsx")


In [17]:
#Create a function which takes a starting position and the length vertically and horizontally
#and then creates borders around all of them
def create_border(ws, start_row, start_column, vertical_length, horizontal_length):
    border = Border(left=Side(border_style='thin'),
         right=Side(border_style='thin'),
         top=Side(border_style='thin'),
         bottom=Side(border_style='thin'))
    for row in range(vertical_length):
        for column in range(horizontal_length):
            ws.cell(row=start_row+row,column=start_column+column).border = border
            
            
wb = openpyxl.Workbook()
ws = wb.active

write_to_excel(ws, 3, 1, yield_curve_table)
create_border(ws, 4, 2, yield_curve_table.shape[0], yield_curve_table.shape[1])

wb.save("Workbooks/Example 9.xlsx")

In [18]:
#Update function
def write_treasury_sheet(ws, yield_curve_table):
    #Write the yield curve table
    write_to_excel(ws, 3, 1, yield_curve_table)
    
    #Color the yield curve changes red/green
    red_green_map(yield_curve_table[["1 Year Change"]], 4, 4)
    red_green_map(yield_curve_table[["5 Year Change"]], 4, 6)
    
    #Adjust the column widths for the table
    adjust_columns(ws,2,[15]*len(yield_curve_table.columns))
    
    #Bold the index and columns
    bold_cells_line(ws, 4, 1, len(yield_curve_table), vertical=True)
    bold_cells_line(ws, 3, 2, len(yield_curve_table.columns), vertical=False)
    
    #Create title
    create_title(ws, 'A1:F1', "Treasury Yield Curve")
    
    #Add borders
    create_border(ws, 4, 2, yield_curve_table.shape[0], yield_curve_table.shape[1])
    
wb = openpyxl.Workbook()
ws = wb.active
write_treasury_sheet(ws, yield_curve_table)
wb.save("Workbooks/Economic Report.xlsx")

/Users/seanmcowen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
  if __name__ == '__main__':


In [19]:
wb = openpyxl.Workbook()
ws = wb.active

write_to_excel(ws, 3, 1, yield_curve_table)
#We might be interested in switching off gridlines
ws.sheet_view.showGridLines = False
wb.save("Workbooks/Example 10.xlsx")

In [20]:
#Updated again
def write_treasury_sheet(ws, yield_curve_table):
    #Write the yield curve table
    write_to_excel(ws, 3, 1, yield_curve_table)
    
    #Color the yield curve changes red/green
    red_green_map(yield_curve_table[["1 Year Change"]], 4, 4)
    red_green_map(yield_curve_table[["5 Year Change"]], 4, 6)
    
    #Adjust the column widths for the table
    adjust_columns(ws,2,[15]*len(yield_curve_table.columns))
    
    #Bold the index and columns
    bold_cells_line(ws, 4, 1, len(yield_curve_table), vertical=True)
    bold_cells_line(ws, 3, 2, len(yield_curve_table.columns), vertical=False)
    
    #Create title
    create_title(ws, 'A1:F1', "Treasury Yield Curve")
    
    #Add borders
    create_border(ws, 4, 2, yield_curve_table.shape[0], yield_curve_table.shape[1])
    
    #Get rid of gridlines
    ws.sheet_view.showGridLines = False
    
wb = openpyxl.Workbook()
ws = wb.active
write_treasury_sheet(ws, yield_curve_table)
wb.save("Workbooks/Economic Report.xlsx")

/Users/seanmcowen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
  if __name__ == '__main__':


In [21]:
from openpyxl.drawing.image import Image

wb = openpyxl.Workbook()
ws = wb.active

write_to_excel(ws, 3, 1, yield_curve_table)

#Images can be put into excel worksheets by loading them then adding image to a given position
img = Image('Images/Treasury Yield Curves.png')
ws.add_image(img, 'H1')

wb.save("Workbooks/Example 11.xlsx")

In [22]:
#Do the final update of the function
def write_treasury_sheet(ws, yield_curve_table):
    #Write the yield curve table
    write_to_excel(ws, 3, 1, yield_curve_table)
    
    #Color the yield curve changes red/green
    red_green_map(yield_curve_table[["1 Year Change"]], 4, 4)
    red_green_map(yield_curve_table[["5 Year Change"]], 4, 6)
    
    #Adjust the column widths for the table
    adjust_columns(ws,2,[15]*len(yield_curve_table.columns))
    
    #Bold the index and columns
    bold_cells_line(ws, 4, 1, len(yield_curve_table), vertical=True)
    bold_cells_line(ws, 3, 2, len(yield_curve_table.columns), vertical=False)
    
    #Create title
    create_title(ws, 'A1:F1', "Treasury Yield Curve")
    
    #Add borders
    create_border(ws, 4, 2, yield_curve_table.shape[0], yield_curve_table.shape[1])
    
    #Get rid of gridlines
    ws.sheet_view.showGridLines = False
    
    #Add yield curve graphs
    img = Image('Images/Treasury Yield Curves.png')
    ws.add_image(img, 'H1')
    
wb = openpyxl.Workbook()
ws = wb.active
write_treasury_sheet(ws, yield_curve_table)
wb.save("Workbooks/Economic Report.xlsx")

/Users/seanmcowen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
  if __name__ == '__main__':
